In [1]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import datetime
import plotly.express as px
from xbbg import blp
import os
import quantstats as qs
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import logging

# Import custom modules with an alias
import bloomberg_data as bd
import transformations as tr
import visuals as vis

In [2]:
# Main data retrieval and merging process
tickers = ['.MIDERCAD U Index', '.CADIG F Index', 'VIX Index', '.HYUSER U Index', '.IGUSER U Index','SPTSX INDEX','ECSURPUS Index','LEI YOY Index','ECRPUS 1Y Index','ECRPCA 1Y Index','GDGCAFJP Index','GDGCAFJP Index','.HARDATA G Index','CGERGLOB Index','.FRCRRM G Index']
fields = [['PX_LAST'], ['PX_LAST'], ['PX_LAST'], ['PX_LAST'], ['PX_LAST'], ['PX_LAST'],['PX_LAST'],['PX_LAST'],['PX_LAST'],['PX_LAST'],['PX_LAST'],['PX_LAST'],['PX_LAST'],['PX_LAST'],['PX_LAST'],['PX_LAST'],['PX_LAST']]
start_date = '2006-01-01'
end_date = '2025-12-31'
column_names = [['cad_ig_er_index'], ['cad_ig_sprds'], ['vix'], ['us_hy_er_index'], ['us_ig_er_index'],['tsx_index'],['us_eco_suprise'],['lei_yoy_index'],['us_recession_odds'],['cad_recession_odds'],['atlanta_fed'],['growth_surpise'],['hard_data'],['equity_revisions'],['fed_credit_model']]
frequency = 'm'  # Single frequency for all tickers

dataframes = []

for ticker, field, col_name in zip(tickers, fields, column_names):
    df = bd.get_single_ticker_data(ticker, field, start_date, end_date, freq=frequency, column_names=col_name)
    dataframes.append(df)
    logging.info(f"Data for {ticker}:")
    logging.info(df.head())  # Print the first few rows of each dataframe

# Merge all dataframes
merged_data = bd.merge_dataframes(dataframes, method='outer')

# Print the final merged data and its information
logging.info("Merged data head:")
logging.info(merged_data.head())
logging.info('----------------------------------------------------------------')
logging.info('----------------------------------------------------------------')
logging.info(merged_data.tail())
logging.info(merged_data.info())

# Rename the index to "Date" and reset it
merged_data.index.name = 'Date'
csv_data = merged_data.reset_index()

# Save the dataframe to a CSV file
csv_data.to_csv('Outputs/csv_data.csv', index=False)

# Rename for further use
data = merged_data

2024-09-22 13:37:27,095 - INFO - Retrieving data for ticker: .MIDERCAD U Index with frequency: MONTHLY
2024-09-22 13:37:27,921 - INFO - Retrieved data shape for .MIDERCAD U Index: (225, 1)
2024-09-22 13:37:27,921 - INFO - Cleaned data shape for .MIDERCAD U Index: (225, 1)
2024-09-22 13:37:27,922 - INFO - Successfully retrieved data for ticker: .MIDERCAD U Index
2024-09-22 13:37:27,922 - INFO - Data for .MIDERCAD U Index:
2024-09-22 13:37:27,923 - INFO -             cad_ig_er_index
2006-01-31           1.0579
2006-02-28           1.0576
2006-03-31           1.0569
2006-04-28           1.0561
2006-05-31           1.0565
2024-09-22 13:37:27,924 - INFO - Retrieving data for ticker: .CADIG F Index with frequency: MONTHLY
2024-09-22 13:37:28,375 - INFO - Retrieved data shape for .CADIG F Index: (225, 1)
2024-09-22 13:37:28,375 - INFO - Cleaned data shape for .CADIG F Index: (225, 1)
2024-09-22 13:37:28,376 - INFO - Successfully retrieved data for ticker: .CADIG F Index
2024-09-22 13:37:28,37

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 289 entries, 2006-01-31 to 2024-09-30
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   cad_ig_er_index     289 non-null    float64
 1   cad_ig_sprds        289 non-null    float64
 2   vix                 289 non-null    float64
 3   us_hy_er_index      289 non-null    float64
 4   us_ig_er_index      289 non-null    float64
 5   tsx_index           289 non-null    float64
 6   us_eco_suprise      289 non-null    float64
 7   lei_yoy_index       289 non-null    float64
 8   us_recession_odds   289 non-null    float64
 9   cad_recession_odds  289 non-null    float64
 10  atlanta_fed         289 non-null    float64
 11  growth_surpise      289 non-null    float64
 12  hard_data           289 non-null    float64
 13  equity_revisions    289 non-null    float64
 14  fed_credit_model    289 non-null    float64
dtypes: float64(15)
memory usage: 36.1 KB


In [3]:
data

,cad_ig_er_index,cad_ig_sprds,vix,us_hy_er_index,us_ig_er_index,tsx_index,us_eco_suprise,lei_yoy_index,us_recession_odds,cad_recession_odds,atlanta_fed,growth_surpise,hard_data,equity_revisions,fed_credit_model
Date,,,,,,,,,,,,,,,
2006-01-31,1.0579,58.7572,12.95,0.5701,1.0877,11945.64,0.048,3.6,70.0,10.0,1.600,1.600,-0.0805,0.15,0.11
2006-02-28,1.0576,59.4640,12.34,0.5731,1.0910,11688.34,0.194,3.1,70.0,10.0,1.600,1.600,0.1658,0.13,0.13
2006-03-31,1.0569,59.6169,11.39,0.5828,1.0874,12110.61,0.221,3.7,70.0,10.0,1.600,1.600,0.0398,0.19,0.11
2006-04-28,1.0561,58.7984,11.59,0.5888,1.0881,12204.17,0.159,3.7,70.0,10.0,1.600,1.600,0.0808,0.27,0.11
2006-04-30,1.0561,58.7984,11.59,0.5888,1.0881,12204.17,0.159,2.6,70.0,10.0,1.600,1.600,0.0808,0.27,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-30,1.3925,120.2679,12.44,1.1270,1.4199,21875.79,-0.477,-4.9,30.0,25.0,2.248,2.248,-0.2787,-0.07,0.20
2024-07-31,1.3932,120.0034,16.36,1.1243,1.4226,23110.81,-0.464,-5.2,30.0,25.0,2.830,2.830,-0.3013,-0.09,0.18
2024-08-30,1.3905,122.7558,15.00,1.1282,1.4267,23346.18,-0.418,-5.2,30.0,25.0,2.531,2.531,-0.2806,-0.15,0.18
